### Cleaning Overview
- Input: Raw GA4 item-level Parquet snapshot
- Objective: Prepare analytics-ready dataset
- Key cleaning steps:
  - Null handling
  - Deduplication
  - Type standardization
  - Feature derivation
- Output: Clean Parquet for downstream analysis


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
use_sample = False

RAW_DATA_PATH = Path('../data/raw')
raw_file = RAW_DATA_PATH / 'fact_products_raw.parquet'

df = pd.read_parquet(raw_file)
df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3839535 entries, 0 to 3839534
Data columns (total 18 columns):
 #   Column           Non-Null Count    Dtype              
---  ------           --------------    -----              
 0   event_date       3839535 non-null  datetime64[ns]     
 1   event_ts         3839535 non-null  datetime64[us, UTC]
 2   event_name       3839535 non-null  object             
 3   user_id          3839535 non-null  object             
 4   transaction_id   14205 non-null    object             
 5   session_id       3839535 non-null  Int64              
 6   device           3839535 non-null  object             
 7   country          3839535 non-null  object             
 8   region           3839535 non-null  object             
 9   city             3839535 non-null  object             
 10  traffic_sname    3839535 non-null  object             
 11  traffic_smedium  3839535 non-null  object             
 12  traffic_source   3839535 non-null  object 